In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import requests
import time
import json

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client.test

In [3]:
def temp_store_memory(agent_id, agents_involved, content):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "content": content, #!!!
    }
    db.agent_memories.insert_one(memory)

In [4]:
temp_store_memory("agent_001",
                  ["agent_002"],
                  "I am cooking pasta.")

In [ ]:
def temp_retrieve_memories(agent_id):
    memories = db.agent_memories.find({
        "agent_id": agent_id
    }).sort("timestamp", -1)
    return list(memories)

def temp_retrieve_memories_concerning(agent_id):
    memories = db.agent_memories.find({
        "$or": [
            {"agent_id": agent_id},
            {"agents_involved": agent_id}
        ]
    }).sort("timestamp", -1)
    return list(memories)

print(temp_retrieve_memories("agent_001"))

[]


In [6]:
class Agent:
    def __init__(self, name, agent_id, user_input, gender):
        self.name = name
        self.agent_id = agent_id 
        self.user_input = user_input
        self.gender = gender
        
    def __str__(self):
        return f"Agent {self.name} (ID: {self.agent_id}) (Description: {self.user_input})"
    
    def name(self):
        return self.name
    
    def agent_id(self):
        return self.agent_id
    
    def user_input(self):
        return self.user_input
    
    def gender(self):
        return self.gender

In [7]:
Sami = Agent("Sami", "agent_001", "Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has a very bad and annoying personality. Other than that, he has an opinion on everything else.", "male")
print(Sami)

Eliana = Agent("Eliana", "agent_002", "Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is a close friend to Gordon Ramsay who taught her everything he knows about cooking. She is the sweetest person on earth and would never lie or attack someone.", "female")
print(Eliana)

Agent Sami (ID: agent_001) (Description: Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has a very bad and annoying personality. Other than that, he has an opinion on everything else.)
Agent Eliana (ID: agent_002) (Description: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is a close friend to Gordon Ramsay who taught her everything he knows about cooking. She is the sweetest person on earth and would never lie or attack someone.)


In [64]:
def make_initial_prompt(agent1, agent2, conversation, subject, memories):
    message_content = f"""
            Context:
            There is two agents.
            Here is a description of {agent1.name}: {agent1}.
            Here is a description of {agent2.name}: {agent2}.
            Here is what {agent1.name} thinks about {agent2.name}: 
            She has good advice for food and is nice but strange.
            Here is what {agent2.name} thinks about {agent1.name}:
            He is cool. She likes his cooking.
            Currently, it is monday morning
            -- Eliana is trying to read
            -- Sami is searching for his next receipe

            The subject of this conversation is: {subject}
            Here is the conversation they had:

            {agent1.name}: "
        """
    return message_content

In [60]:
url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}

def chat(agent1, agent2, subject):
    conversation = []
    # First agent's turn
    memories = temp_retrieve_memories(agent1.agent_id)
    
    message_prompt = make_initial_prompt(agent2, agent1, conversation, subject, memories)

    data = {
            "model": "llama-3.2-1b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": message_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }

    response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
    print(response)
    temp_store_memory(agent1.agent_id, [agent2.agent_id], "Had this conversation with " + agent2.name + ": " + str(response))
    temp_store_memory(agent2.agent_id, [agent1.agent_id], "Had this conversation with " + agent1.name + ": " + str(response))

In [63]:
chat(Sami, Eliana, "They are talking about food")

Eliana: "Monday morning, isn't it? I'm so excited to try out a new recipe today. I found this one for some kind of Italian dish that Sami recommended last night. Do you think I should give it a shot?"

Sami: "Ahah, Eliana! You're always trying my recipes and eating them up like they're your own cooking! That's what I love about our conversations - the food is just as delicious as my cookbooks!"
